In [ ]:
!pip install opencv-python
!pip install ultralytics

In [111]:
import cv2
import numpy as np
import os

In [130]:
def rotation(img1n, img2n):
    img1 = cv2.imread(img1n)
    img2 = cv2.imread(img2n)
    #grayscale
    g1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    g2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    #oriented BRIEF
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(g1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(g2, None)

    #brute force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    #sorting matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    #get matched keypoints
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    #homography matrix
    homo, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    #finally get rotation angle from matrix
    rotation_angle = np.arctan2(homo[1, 0], homo[0, 0]) * 180 / np.pi

    #writing rotation angle on the image
    img = cv2.putText(img2, str(int(rotation_angle))+"deg", org = (150, 500), fontFace=2, fontScale = 3, color = (0, 0, 255), thickness = 2)
    #the folders are already created so just using "/" else need to create them
    cv2.imwrite("C:/Users/Rugved/GitHub/mowito/output/"+img1n[-34:-4]+'/'+img2n[-34:], img)

    return rotation_angle


In [129]:
img1n = 'C:/Users/Rugved/GitHub/mowito/data new/template/screenshot_2024-05-02_17-22-50.jpg'
img2n = 'C:/Users/Rugved/GitHub/mowito/data new/test/screenshot_2024-05-02_17-23-39.jpg'

rotation_angle = rotation(img1n, img2n)
print("Rotation angle is", int(rotation_angle))

Rotation angle is -114


In [131]:
#getting outputs for all test images against all templates
for i in os.listdir('C:/Users/Rugved/GitHub/mowito/data new/template/'):
    for j in os.listdir('C:/Users/Rugved/GitHub/mowito/data new/test/'):
        rotation_angle = rotation('C:/Users/Rugved/GitHub/mowito/data new/template/'+i, 'C:/Users/Rugved/GitHub/mowito/data new/test/'+j)

In [ ]:
#using pretrained yolov8 segmentation model to crop the image accurately

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-seg.pt')

100%|██████████| 6.73M/6.73M [00:00<00:00, 10.4MB/s]


In [37]:
template_image = 'screenshot_2024-05-02_17-18-27.jpg'
result = model.predict('C:/Users/Rugved/GitHub/mowito/data/template images/'+template_image, save=True, save_crop=True, show_labels=False, project="bounding_box_output", name=template_image)


image 1/1 C:\Users\Rugved\GitHub\mowito\data\template images\screenshot_2024-05-02_17-18-27.jpg: 384x640 1 teddy bear, 77.8ms
Speed: 0.0ms preprocess, 77.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to output\screenshot_2024-05-02_17-18-27.jpg


In [26]:
for r in result:
    print(r.tojson())

[
  {
    "name": "teddy bear",
    "class": 77,
    "confidence": 0.40916,
    "box": {
      "x1": 759.55957,
      "y1": 84.3479,
      "x2": 1580.23096,
      "y2": 865.50092
    },
    "segments": {
      "x": [
        1332.0,
        1329.0,
        1317.0,
        1311.0,
        1308.0,
        1302.0,
        1299.0,
        1284.0,
        1284.0,
        1275.0,
        1275.0,
        1272.0,
        1272.0,
        1269.0,
        1269.0,
        1266.0,
        1266.0,
        1263.0,
        1263.0,
        1260.0,
        1260.0,
        1257.0,
        1257.0,
        1254.0,
        1254.0,
        1248.0,
        1248.0,
        1242.0,
        1239.0,
        1233.0,
        1230.0,
        1224.0,
        1221.0,
        1218.0,
        1215.0,
        1215.0,
        1209.0,
        1206.0,
        1203.0,
        1188.0,
        1185.0,
        1179.0,
        1176.0,
        1170.0,
        1167.0,
        1158.0,
        1155.0,
        1152.0,
        1137.0,

In [116]:
for i in os.listdir('C:/Users/Rugved/GitHub/mowito/data/template images/'):
    model.predict('C:/Users/Rugved/GitHub/mowito/data/template images/'+i, save=True, save_crop=True, show_labels=False, project="bounding_box_output_template", name=i)


image 1/1 C:\Users\Rugved\GitHub\mowito\data\template images\screenshot_2024-05-02_17-12-17.jpg: 384x640 1 teddy bear, 60.4ms
Speed: 2.0ms preprocess, 60.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_template\screenshot_2024-05-02_17-12-17.jpg

image 1/1 C:\Users\Rugved\GitHub\mowito\data\template images\screenshot_2024-05-02_17-18-27.jpg: 384x640 1 teddy bear, 52.9ms
Speed: 0.0ms preprocess, 52.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_template\screenshot_2024-05-02_17-18-27.jpg

image 1/1 C:\Users\Rugved\GitHub\mowito\data\template images\screenshot_2024-05-02_17-22-50.jpg: 384x640 1 airplane, 1 kite, 55.0ms
Speed: 1.4ms preprocess, 55.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_template\screenshot_2024-05-02_17-22-50.jpg


In [117]:
for i in os.listdir('C:/Users/Rugved/GitHub/mowito/data/test images/'):
    model.predict('C:/Users/Rugved/GitHub/mowito/data/test images/'+i, save=True, save_crop=True, show_labels=False, project="bounding_box_output_test", name=i)


image 1/1 C:\Users\Rugved\GitHub\mowito\data\test images\screenshot_2024-05-02_17-11-22.jpg: 384x640 1 motorcycle, 1 airplane, 57.6ms
Speed: 2.0ms preprocess, 57.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_test\screenshot_2024-05-02_17-11-22.jpg

image 1/1 C:\Users\Rugved\GitHub\mowito\data\test images\screenshot_2024-05-02_17-12-08.jpg: 384x640 (no detections), 57.2ms
Speed: 0.0ms preprocess, 57.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_test\screenshot_2024-05-02_17-12-08.jpg

image 1/1 C:\Users\Rugved\GitHub\mowito\data\test images\screenshot_2024-05-02_17-12-30.jpg: 384x640 1 teddy bear, 54.5ms
Speed: 3.6ms preprocess, 54.5ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to bounding_box_output_test\screenshot_2024-05-02_17-12-30.jpg

image 1/1 C:\Users\Rugved\GitHub\mowito\data\test images\screenshot_2024-05-02_17-12-40.jpg: 